In [1]:
# pridam si src slozku do cesty
import sys
import os
pwd = %pwd
sys.path.append(os.path.join('src'))

%load_ext autoreload
%autoreload 2

In [2]:
# knihovny
import numpy as np

In [3]:
# nactu si Rastrigin
from objfun_rastrigin_Lorenz import Rastrigin

In [4]:
rast = Rastrigin(n=3,eps=1.0)

In [5]:
x = rast.generate_point()
print(x)
print(rast.evaluate(x))

[ 1.9112086  -1.77425184 -3.82513427]
36.88330332543175


In [6]:
# Import external librarires
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [7]:
# GENETICKA OPTIMALIZACE Z HODINY:
# prvni zkouska:
from heur_go import GeneticOptimization
from heur_aux import Correction, MirrorCorrection, ExtensionCorrection, CauchyMutation
from heur_go import Crossover, UniformMultipoint, RandomCombination
cauchy = CauchyMutation(r=.75, correction=MirrorCorrection(rast))
heur_1 = GeneticOptimization(rast, maxeval=100000, N=10, M=30, Tsel1=0.5, Tsel2=0.1, 
                           mutation=cauchy, crossover=RandomCombination())
res = heur_1.search()
print(res)

{'best_y': 0.3126217407426246, 'best_x': array([-0.01299789,  0.014027  , -0.03485858]), 'neval': 69426, 'log_data': Empty DataFrame
Columns: []
Index: []}


In [8]:
NUM_RUNS = 10 # lze potom zvysit
maxeval = 50000 # nejde moc snizit v pripade skolni implementace GO

In [9]:
def experiment_go(of, maxeval, num_runs, N, M, Tsel1, Tsel2, mutation, crossover):
    results = []
    heur_name = 'GO_{}'.format(N)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = GeneticOptimization(of, maxeval, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover).search()
        result['run'] = i
        result['heur'] = heur_name
        result['N'] = N
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'N', 'best_x', 'best_y', 'neval'])

In [10]:
results = pd.DataFrame()
mutation = CauchyMutation(r=0.1, correction=MirrorCorrection(rast))
crossover = RandomCombination()
for N in [1, 2, 3, 5, 10, 20, 30, 100]:
    res = experiment_go(of=rast, maxeval=maxeval, num_runs=NUM_RUNS, N=N, M=N*3, Tsel1=0.1, Tsel2=1, 
                        mutation=mutation, crossover=crossover)
    results = pd.concat([results, res], axis=0)

In [11]:
def rel(x): # REL=m/q where m is number of successful runs and q is total number of runs
    return len([n for n in x if n < np.inf])/len(x)
def mne(x): # Mean Number of objective function Evaluations in the succesful runs
    return np.mean([n for n in x if n < np.inf])
def feo(x): # eoktistov criterion: FEO = MNE/REL
    return mne(x)/rel(x)

In [12]:
results_pivot = results.pivot_table(
    index=['heur', 'N'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot = results_pivot.reset_index()
results_pivot.sort_values(by='N')
# vidime tu ze velky pocet behu je potreba (cca maxeval>35000)

,heur,N,feo,mne,rel
0,GO_1,1,36599.2,18299.6,0.5
3,GO_2,2,5913.2,5913.2,1.0
5,GO_3,3,3405.3,3405.3,1.0
7,GO_5,5,1439.0,1439.0,1.0
1,GO_10,10,2793.4,2793.4,1.0
4,GO_20,20,2709.3,2709.3,1.0
6,GO_30,30,2601.3,2601.3,1.0
2,GO_100,100,2448.7,2448.7,1.0


In [13]:
# MOJE IMPLEMENTACE:
from heur_aux import Correction, MirrorCorrection, ExtensionCorrection, CauchyMutation
from heur_go_Lorenz import GeneticOptimization, Crossover, MeanChild
cauchy = CauchyMutation(r=0.1, correction=MirrorCorrection(rast))
heur_2 = GeneticOptimization(rast, maxeval=100000, N=10, M=30, Tsel1=0.5, Tsel2=0.1, 
                           mutation=cauchy, crossover=MeanChild())
res2 = heur_2.search()
print(res2)

{'best_y': 0.2193276072206629, 'best_x': array([-0.00363884, -0.00756403,  0.03222744]), 'neval': 3141, 'log_data': Empty DataFrame
Columns: []
Index: []}


In [14]:
results2 = pd.DataFrame()
mutation = CauchyMutation(r=0.75, correction=MirrorCorrection(rast))
crossover = MeanChild()
for N in [1, 2, 3, 5, 10, 20, 30, 100]:
    res2 = experiment_go(of=rast, maxeval=maxeval, num_runs=NUM_RUNS, N=N, M=N*3, Tsel1=0.1, Tsel2=1, 
                        mutation=mutation, crossover=crossover)
    results2 = pd.concat([results2, res2], axis=0)

In [15]:
results_pivot2 = results2.pivot_table(
    index=['heur', 'N'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot2 = results_pivot2.reset_index()
results_pivot2.sort_values(by='N')
# nevypada to ze se to nejak zlepsilo,
#ale ponecham to

,heur,N,feo,mne,rel
0,GO_1,1,163312.500000,32662.500000,0.2
3,GO_2,2,63972.000000,31986.000000,0.5
5,GO_3,3,52930.625000,21172.250000,0.4
7,GO_5,5,59825.625000,23930.250000,0.4
1,GO_10,10,24674.081633,17271.857143,0.7
4,GO_20,20,39055.200000,19527.600000,0.5
6,GO_30,30,20500.123457,18450.111111,0.9
2,GO_100,100,29581.224490,20706.857143,0.7


In [16]:
def experiment_go_rr(of, maxeval, num_runs, N, M, Tsel1, Tsel2, mutation, crossover):
    results = []
    heur_name = 'GO_{}'.format(rr)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = GeneticOptimization(of, maxeval, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover).search()
        result['run'] = i
        result['heur'] = heur_name
        result['N'] = N
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'N', 'best_x', 'best_y', 'neval'])

In [17]:
# jeste RESEARCH NA PARAMETR R V CAUCHY MUTATION:
results3 = pd.DataFrame()
crossover = MeanChild()
N = 10;
for rr in [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2]:
    mutation = CauchyMutation(r=rr, correction=MirrorCorrection(rast))
    res3 = experiment_go_rr(of=rast, maxeval=maxeval, num_runs=NUM_RUNS, N=N, M=N*3, Tsel1=0.1, Tsel2=1, 
                        mutation=mutation, crossover=crossover)
    results3 = pd.concat([results3, res3], axis=0)

In [18]:
results_pivot3 = results3.pivot_table(
    index=['heur', 'N'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot3 = results_pivot3.reset_index()
results_pivot3.sort_values(by='N')
# to vypada dobre, zvlast r=0.1

,heur,N,feo,mne,rel
0,GO_0.01,10,2972.100000,2972.100000,1.0
1,GO_0.02,10,2664.800000,2664.800000,1.0
2,GO_0.05,10,2651.600000,2651.600000,1.0
3,GO_0.1,10,2144.500000,2144.500000,1.0
4,GO_0.2,10,2404.300000,2404.300000,1.0
5,GO_0.5,10,22084.567901,19876.111111,0.9
6,GO_1,10,54427.600000,27213.800000,0.5
7,GO_2,10,347430.000000,34743.000000,0.1


'<heur_go_Lorenz.MeanChild object at 0x000001A7C6FD90B8>'